In [1]:
from ophyd import Device, EpicsMotor, EpicsSignal, EpicsSignalRO
from ophyd import Component as Cpt
import time
import pandas as pd
import numpy as np
from ophyd.sim import det4, noisy_det, motor  # simulated detector, motor
import h5py
from datetime import datetime

In [9]:
from bluesky import RunEngine
import bluesky.plan_stubs as bps
from bluesky.plans import count, scan
RE = RunEngine({})

In [11]:
## Prepare Live Visualization
from bluesky.callbacks.best_effort import BestEffortCallback
bec = BestEffortCallback()

# Send all metadata/data captured to the BestEffortCallback.
RE.subscribe(bec)

# Make plots update live while scans run.
from bluesky.utils import install_kicker
install_kicker()
#%matplotlib notebook
#from bluesky.utils import install_nb_kicker
#install_nb_kicker()


## Prepare Data Storage
from databroker import Broker
db = Broker.named('xpd')

# Insert all metadata/data captured into db.
RE.subscribe(db.insert)


## Add a Progress Bar
from bluesky.utils import ProgressBarManager
RE.waiting_hook = ProgressBarManager()


/nsls2/conda/envs/2022-3.0-py39-tiled/lib/python3.9/site-packages/bluesky/utils/__init__.py:932: UserWarning: bluesky.utils.install_qt_kicker is no longer necessary and has no effect. Please remove your use of it. It may be removed in a future release of bluesky.
  warnings.warn("bluesky.utils.install_qt_kicker is no longer necessary and "
OBJECT CACHE: Will use up to 40_490_850_508 bytes (15% of total physical RAM)
INFO:tiled.server.object_cache:Will use up to 40_490_850_508 bytes (15% of total physical RAM)


In [35]:
RE(count([det4], num=5), md)



Transient Scan ID: 2     Time: 2022-10-13 16:36:00
Persistent Unique Scan ID: '8526f12c-89c7-4f0f-872a-9f75a552e0a2'


Transient Scan ID: 2     Time: 2022-10-13 16:36:00
Persistent Unique Scan ID: '8526f12c-89c7-4f0f-872a-9f75a552e0a2'
New stream: 'primary'
+-----------+------------+------------+
|   seq_num |       time |       det4 |
+-----------+------------+------------+
New stream: 'primary'
+-----------+------------+------------+
|   seq_num |       time |       det4 |
+-----------+------------+------------+
|         1 | 16:36:00.1 |      1.000 |
|         1 | 16:36:00.1 |      1.000 |
|         2 | 16:36:00.1 |      1.000 |
|         2 | 16:36:00.1 |      1.000 |
|         3 | 16:36:00.1 |      1.000 |
|         3 | 16:36:00.1 |      1.000 |
|         4 | 16:36:00.1 |      1.000 |
|         4 | 16:36:00.1 |      1.000 |
|         5 | 16:36:00.1 |      1.000 |
|         5 | 16:36:00.1 |      1.000 |
|         6 | 16:36:00.1 |      1.000 |
|         6 | 16:36:00.1 |      1.000 

('8526f12c-89c7-4f0f-872a-9f75a552e0a2',)

In [73]:
db[-1].header

AttributeError: 'Header' object has no attribute 'header'

In [69]:
## Available pump list : [dds1_p1, dds1_p2, dds2_p1, dds2_p2, ultra1, ultra2]
pump_list = [dds1_p1, dds1_p2]
target_vol_list = ['50 ml', '50 ml']
infuse_rate_list = ['100 ul/min', '100 ul/min']
withdraw_rate_list = ['100 ul/min', '100 ul/min']


def reset_pumps(pump_list, clear=True):
    for pump in pump_list:
        pump.initialize_pump(clear=clear)
        pump.infuse_rate_unit.put('ul/min', wait=True)
        pump.infuse_rate.put(100, wait=True)
        pump.withdraw_rate_unit.put('ul/min', wait=True)
        pump.withdraw_rate.put(100, wait=True)
        pump.target_vol_unit.put('ml', wait=True)
        pump.target_vol.put(20, wait=True)


def show_pump_status(pump_list, precursor_list, wait=False):
    for pump, precursor in zip(pump_list, precursor_list):
        print('Name: ' + f'{pump.name}')
        print('Precursor: ' + f'{precursor}')
        pump.check_pump_condition(wait=wait)
        print('\n')
        

def set_infuse_rates(pump_list, target_vol_list=['50 ml', '50 ml'], rate_list = ['100 ul/min', '100 ul/min']):
    for i, j, k in zip(pump_list, target_vol_list, infuse_rate_list):
        vol = float(j.split(' ')[0])
        vol_unit = j.split(' ')[1]
        rate = float(k.split(' ')[0])
        rate_unit = k.split(' ')[1]        
        i.set_pump(target_vol = vol, target_unit = vol_unit, infuse_rate = rate, infuse_unit = rate_unit)
        
def set_withdraw_rates(pump_list, target_vol_list=['50 ml', '50 ml'], rate_list = ['100 ul/min', '100 ul/min']):
    for i, j, k in zip(pump_list, target_vol_list, rate_list):
        vol = float(j.split(' ')[0])
        vol_unit = j.split(' ')[1]
        rate = float(k.split(' ')[0])
        rate_unit = k.split(' ')[1]        
        i.set_pump(target_vol = vol, target_unit = vol_unit, withdraw_rate = rate, withdraw_unit = rate_unit)



In [62]:
pump_list = [dds1_p1, dds1_p2]
target_vol_list = ['22 ml', '36 ml']
infuse_rate_list = ['101 ul/min', '222 ul/min']
precursor_list = ['CsPb 30 mM', 'TetraBr 30 mM']

#reset_pumps(pump_list)
set_infuse_rates(pump_list, target_vol_list=target_vol_list, rate_list=infuse_rate_list)

In [70]:
show_pump_status(pump_list, precursor_list)

Name: DDS1_p1
Precursor: CsPb 30 mM
Syringe Volume: 50 mL steel syringe
Pump Stauts: Idle
Cmmunication: Enabled
Target Volume: 22.0 ml
Infuse rate: 101.0 ul/min
Withdraw rate: 100.0 ul/min
Infused volume: 0.0 ul
Withdrawn volume: 0.0 ul


Name: DDS1_p2
Precursor: TetraBr 30 mM
Syringe Volume: 50 mL steel syringe
Pump Stauts: Idle
Cmmunication: Enabled
Target Volume: 36.0 ml
Infuse rate: 222.0 ul/min
Withdraw rate: 100.0 ul/min
Infused volume: 0.0 ul
Withdrawn volume: 0.0 ul




In [74]:
RE(dds1_p1.infuse_pump())

ERROR:bluesky:Run aborted
Traceback (most recent call last):
  File "/nsls2/conda/envs/2022-3.0-py39-tiled/lib/python3.9/site-packages/bluesky/run_engine.py", line 1559, in _run
    msg.command, msg.obj, msg.args, msg.kwargs,
AttributeError: 'str' object has no attribute 'command'


AttributeError: 'str' object has no attribute 'command'

In [75]:
db[-1]

In [36]:
qepro.set_temp(-5)

Reached setpoint -5.0.


In [37]:
qepro.setup_collection?

Signature:
qepro.setup_collection(
    integration_time=100,
    num_spectra_to_average=10,
    spectrum_type='Absorbtion',
    correction_type='Reference',
    electric_dark_correction=True,
)
Docstring: <no docstring>
File:      /tmp/ipykernel_1287621/692090166.py
Type:      method


In [64]:
qepro.setup_collection(integration_time=100, num_spectra_to_average=10, electric_dark_correction=True)

In [60]:
shutter_close()
qepro.get_dark_frame()
shutter_open()

'High'

In [61]:
qepro.get_reference_frame()

In [65]:
qepro.trigger()

In [67]:
qepro.plot_spectra()

In [239]:
def h5_rate_spectra(h5_path, h5_name, pump_list, precursor_list, absor=qepro, xray=det4, delay=None):

    fout = h5_path + h5_name
    t_q = list(absor.read().items())[0][1]['timestamp']
    dt = datetime.fromtimestamp(t_q)

    
    with h5py.File(fout, 'w') as hf:
        hf.create_dataset('Time_QEPro', data=[f'{dt.year}{dt.month:02d}{dt.day:02d}', f'{dt.hour:02d}{dt.minute:02d}{dt.second:02d}'])
        hf.create_dataset('Wavelength', data=absor.x_axis.get())
        hf.create_dataset('Dark', data=absor.dark.get())
        if absor.spectrum_type.get(as_string=True) == 'Absorbtion':           
            hf.create_dataset('Reference', data=absor.reference.get())
            hf.create_dataset('Sample_UV', data=absor.sample.get())
            hf.create_dataset('Abosrbance', data=absor.output.get())
        else:
            hf.create_dataset('Sample_PL', data=absor.sample.get())
            hf.create_dataset('PL', data=absor.output.get())            
        for pump, precursor in zip(pump_list, precursor_list):
            hf.create_dataset(pump.name, data=[f'{precursor}', f'{pump.read_infuse_rate.get()}', 
                                               pump.read_infuse_rate_unit.get()])      
    print(f'{fout} is saved.')
    hf.close()

    

    
def csv_rate_spectra(csv_path, csv_name, pump_list, precursor_list, absor=qepro, xray=det4, delay=None):

    fout = csv_path + csv_name
    t_q = list(absor.read().items())[0][1]['timestamp']
    dt = datetime.fromtimestamp(t_q)
    
    with open(fout, 'w') as fp:
        for pump, precursor in zip(pump_list, precursor_list):
            fp.write(f'{pump.name},{precursor},{pump.read_infuse_rate.get()},{pump.read_infuse_rate_unit.get()}\n')        
        
        fp.write(f'Time_QEPro,{dt.year}{dt.month:02d}{dt.day:02d},{dt.hour:02d}{dt.minute:02d}{dt.second:02d}\n')
        x_axis_data = absor.x_axis.get()
        output_data = absor.output.get()
        sample_data = absor.sample.get()
        dark_data = absor.dark.get()
        reference_data = absor.reference.get()
        if absor.spectrum_type.get(as_string=True) == 'Absorbtion':
            fp.write('Energy,Dark,Reference,Sample,Absorbance\n')
        else:
            fp.write('Energy,Dark,Raw Sample,PL\n')

        for i in range(len(output_data)):
            if absor.spectrum_type.get(as_string=True) == 'Absorbtion':
                fp.write(f'{x_axis_data[i]},{dark_data[i]},{reference_data[i]},{sample_data[i]},{output_data[i]}\n')
            else:
                fp.write(f'{x_axis_data[i]},{dark_data[i]},{sample_data[i]},{output_data[i]}\n')

        print('Done.')


In [241]:
h5_path = '/nsls2/users/clin1/Documents/Cheng-Hung/temp/'
h5_name = '20220912_test6.h5'
h5_rate_spectra(h5_path, h5_name, pump_list, precursor_list, absor=qepro)

/nsls2/users/clin1/Documents/Cheng-Hung/temp/20220912_test6.h5 is saved.


In [242]:
csv_path = '/nsls2/users/clin1/Documents/Cheng-Hung/temp/'
csv_name = '20220912_test3.csv'
csv_rate_spectra(csv_path, csv_name, pump_list, precursor_list, absor=qepro)

Done.


In [238]:
f = '/nsls2/users/clin1/Documents/Cheng-Hung/temp/20220912_test2.csv'
df = pd.read_csv(f, skiprows=3)
df

,Energy,Dark,Reference,Sample,Absorption
0,196.262863,13.375,1.875,11.975,NaN
1,197.066232,0.375,-9.125,-1.925,NaN
2,197.869529,2.375,-12.125,-6.025,0.247821
3,198.672754,-4.625,-18.125,-6.025,NaN
4,199.475907,2.375,711.875,101.975,0.853046
...,...,...,...,...,...
1039,989.418604,2.375,157033.875,27225.275,0.761054
1040,990.139347,-2.625,45.875,6.475,0.732903
1041,990.860002,-4.625,1.875,-0.425,NaN
1042,991.580571,0.375,-5.125,-1.525,0.591914


In [127]:
b = list(dds1_p1.read().items())
b[0][1]['timestamp']
dt = datetime.fromtimestamp(b[0][1]['timestamp'])
dt?

Type:        datetime
String form: 2022-09-12 17:47:44.002734
File:        /nsls2/conda/envs/2022-3.0-py39-tiled/lib/python3.9/datetime.py
Docstring:  
datetime(year, month, day[, hour[, minute[, second[, microsecond[,tzinfo]]]]])

The year, month and day arguments are required. tzinfo may be None, or an
instance of a tzinfo subclass. The remaining arguments may be ints.


In [201]:
dds1_p1.name

'DDS1_p1'

In [203]:
for pump, precursor in zip(pump_list, precursor_list):
    print(pump.name)
    data=[f'{precursor}', pump.read_infuse_rate.get(), pump.read_infuse_rate_unit.get()]
    print(data)

DDS1_p1
['CsPb 30 mM', 76.0, 'ul/min']
DDS1_p2
['TetraBr 30 mM', 20.0, 'ul/min']
